#LIBRARY IMPORT

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

#필요한 함수정의

In [3]:
# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

#데이터 불러오기

In [4]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/LG AI대회/'

Mounted at /content/drive


In [13]:
#데이터 모두 불러옴

train_err  = pd.read_csv(PATH+'train_err_data.csv')

test_err  = pd.read_csv(PATH+'test_err_data.csv')

train_quality  = pd.read_csv(PATH+'train_quality_data.csv')

test_quality  = pd.read_csv(PATH+'test_quality_data.csv')

#TRAIN 변수 만들기(DataFrame)

In [7]:
#train_quality데이터 셋에 연,월,일,시 변수 만들어 주기
train_quality['datetime'] = train_quality['time'].apply(make_datetime)
train_quality['day'] = train_quality.loc[:,'datetime'].dt.day
train_quality['month'] = train_quality.loc[:,'datetime'].dt.month
train_quality['year'] = train_quality.loc[:,'datetime'].dt.year
train_quality['hour'] = train_quality.loc[:,'datetime'].dt.hour

In [8]:
quality = train_quality
quality.info()
quality.fillna(0) #quality_5변수에 NA값이 있음

#quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
        'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
quality[cols] = quality[cols].replace({',':''}, regex= True) #2,345이런식으로 표현된(문자열) 원소들이 있음
quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        828624 non-null  int64         
 1   user_id     828624 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   684192 non-null  float64       
 4   quality_1   828624 non-null  int64         
 5   quality_2   788511 non-null  float64       
 6   quality_3   828624 non-null  int64         
 7   quality_4   828624 non-null  int64         
 8   quality_5   828604 non-null  object        
 9   quality_6   828624 non-null  int64         
 10  quality_7   828624 non-null  object        
 11  quality_8   828624 non-null  object        
 12  quality_9   828624 non-null  object        
 13  quality_10  828624 non-null  object        
 14  quality_11  828624 non-null  int64         
 15  quality_12  828624 non-null  int64         
 16  da

In [9]:
#quality1만들기(quality의 일별 합)

quality = quality.drop_duplicates() #중복되는 행 모두제거(모든열에 대해서)

quality1 = quality.groupby(['user_id','month','day'])[cols].max() #규칙을찾음(무조건은 아님)
#규칙: quality변수중 가장 큰 원소값(max)을 가지는것이 그 시간대의 누적quality합임

qsum = []
#위 규칙대로 각 개체별 최대값을 qsum이라는 변수로 만듦
for i in range(0,len(quality1)):
  qsum.append(quality1.iloc[i].max())
quality1['qsum'] = qsum
quality1.head()

quality_0  quality_1  ...  quality_12  qsum
user_id month day                        ...                  
10000   11    29         0.0        0.0  ...         0.0   4.0
              30         0.0        0.0  ...         0.0   8.0
10002   11    4          2.0        0.0  ...         0.0   2.0
              6          0.0        0.0  ...         0.0   2.0
              11         0.0        0.0  ...         0.0   4.0

[5 rows x 14 columns]

In [10]:
quality1 = quality1.reset_index() #인덱스들을 변수로 빼내기

#index = user_id , columns = [month,day] , value = quality발생합 인 피벗테이블 만들기
#피벗테이블 만들기
quality1 = pd.pivot_table(quality1, values = 'qsum', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
quality1.fillna(0, inplace=True) #na를 0으로 채움
quality1.head()

month     10   11                            ...                                   
day       31   1    2    3     4    5    6   ...   24   25   26   27   28   29   30
user_id                                      ...                                   
10000    0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  4.0  8.0
10002    0.0  0.0  0.0  0.0   2.0  0.0  2.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  8.0
10004    0.0  0.0  3.0  0.0  87.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
10005    0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0
10006    0.0  0.0  0.0  0.0   0.0  0.0  0.0  ...  0.0  0.0  0.0  4.0  0.0  0.0  0.0

[5 rows x 31 columns]

In [11]:
#quality2만들기(quality로그 자체가 발생한 일별 빈도수)

quality2 = train_quality
quality2['count'] = 1

#index = user_id , columns = [month,day] , value = quality로그자체발생빈도수 인 피벗테이블 만들기
#피벗테이블 만들기
quality2 = pd.pivot_table(quality2, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
quality2.fillna(0,inplace=True)
quality2.head()

month     10   11                        ...                                 
day       31   1     2    3     4    5   ...   25   26    27   28    29    30
user_id                                  ...                                 
10000    0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0  0.0   0.0  0.0  12.0  12.0
10002    0.0  0.0   0.0  0.0  12.0  0.0  ...  0.0  0.0   0.0  0.0   0.0  24.0
10004    0.0  0.0  12.0  0.0  12.0  0.0  ...  0.0  0.0   0.0  0.0   0.0   0.0
10005    0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0  0.0   0.0  0.0   0.0   0.0
10006    0.0  0.0   0.0  0.0   0.0  0.0  ...  0.0  0.0  12.0  0.0   0.0   0.0

[5 rows x 31 columns]

In [12]:
print(quality1.shape, quality2.shape)

#추후에left조인(by: user_id)을 이용하고 fillna(0)을 이용

(8281, 31) (8281, 31)


#TEST변수 만들기(DataFrame)

In [14]:
#test_quality데이터 셋에 연,월,일,시 변수 만들어 주기
test_quality['datetime'] = test_quality['time'].apply(make_datetime)
test_quality['day'] = test_quality.loc[:,'datetime'].dt.day
test_quality['month'] = test_quality.loc[:,'datetime'].dt.month
test_quality['year'] = test_quality.loc[:,'datetime'].dt.year
test_quality['hour'] = test_quality.loc[:,'datetime'].dt.hour

In [15]:
quality = test_quality
quality.info()
quality.fillna(0) #quality_5변수에 NA값이 있음

#quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
        'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
quality[cols] = quality[cols].replace({',':''}, regex= True) #2,345이런식으로 표현된(문자열) 원소들이 있음
quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747972 entries, 0 to 747971
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        747972 non-null  int64         
 1   user_id     747972 non-null  int64         
 2   fwver       725208 non-null  object        
 3   quality_0   641388 non-null  float64       
 4   quality_1   747961 non-null  object        
 5   quality_2   726857 non-null  float64       
 6   quality_3   747972 non-null  int64         
 7   quality_4   747972 non-null  int64         
 8   quality_5   747928 non-null  object        
 9   quality_6   747972 non-null  int64         
 10  quality_7   747972 non-null  object        
 11  quality_8   747972 non-null  object        
 12  quality_9   747972 non-null  object        
 13  quality_10  747972 non-null  object        
 14  quality_11  747972 non-null  int64         
 15  quality_12  747972 non-null  int64         
 16  da

In [16]:
#quality3만들기(quality의 일별 합)

quality = quality.drop_duplicates() #중복되는 행 모두제거(모든열에 대해서)

quality3 = quality.groupby(['user_id','month','day'])[cols].max() #규칙을찾음(무조건은 아님)
#규칙: quality변수중 가장 큰 원소값(max)을 가지는것이 그 시간대의 누적quality합임

qsum = []
#위 규칙대로 각 개체별 최대값을 qsum이라는 변수로 만듦
for i in range(0,len(quality3)):
  qsum.append(quality3.iloc[i].max())
quality3['qsum'] = qsum
quality3.head()

quality_0  quality_1  ...  quality_12    qsum
user_id month day                        ...                    
30000   11    28         0.0        0.0  ...         0.0     5.0
30001   11    2          0.0        0.0  ...         0.0     3.0
              4          0.0        0.0  ...         0.0     4.0
              12         0.0        0.0  ...         0.0  2556.0
30002   11    1          0.0        0.0  ...         0.0     6.0

[5 rows x 14 columns]

In [17]:
quality3 = quality3.reset_index() #인덱스들을 변수로 빼내기

#index = user_id , columns = [month,day] , value = quality발생합 인 피벗테이블 만들기
#피벗테이블 만들기
quality3 = pd.pivot_table(quality3, values = 'qsum', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
quality3.fillna(0, inplace=True) #na를 0으로 채움
quality3.head()

month     10   11                      ...                                
day       31   1    2    3    4    5   ...    25    26   27   28   29   30
user_id                                ...                                
30000    0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.0  5.0  0.0  0.0
30001    0.0  0.0  3.0  0.0  4.0  0.0  ...   0.0   0.0  0.0  0.0  0.0  0.0
30002    0.0  6.0  0.0  3.0  0.0  0.0  ...   0.0   0.0  0.0  0.0  0.0  0.0
30003    0.0  0.0  0.0  0.0  0.0  0.0  ...  60.0   0.0  0.0  0.0  0.0  0.0
30004    0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0  24.0  0.0  0.0  0.0  0.0

[5 rows x 31 columns]

In [18]:
#quality4만들기(quality로그 자체가 발생한 일별 빈도수)

quality4 = test_quality
quality4['count'] = 1

#index = user_id , columns = [month,day] , value = quality로그자체발생빈도수 인 피벗테이블 만들기
#피벗테이블 만들기
quality4 = pd.pivot_table(quality4, values = 'count', index = 'user_id', columns = ['month','day'],aggfunc = 'sum')
quality4.fillna(0,inplace=True)
quality4.head()

month     10    11                         ...                                 
day       31    1     2     3     4    5   ...    25    26   27    28   29   30
user_id                                    ...                                 
30000    0.0   0.0   0.0   0.0   0.0  0.0  ...   0.0   0.0  0.0  12.0  0.0  0.0
30001    0.0   0.0  12.0   0.0  12.0  0.0  ...   0.0   0.0  0.0   0.0  0.0  0.0
30002    0.0  12.0   0.0  12.0   0.0  0.0  ...   0.0   0.0  0.0   0.0  0.0  0.0
30003    0.0   0.0   0.0   0.0   0.0  0.0  ...  12.0   0.0  0.0   0.0  0.0  0.0
30004    0.0   0.0   0.0   0.0   0.0  0.0  ...   0.0  12.0  0.0   0.0  0.0  0.0

[5 rows x 31 columns]

In [20]:
print(quality3.shape, quality4.shape)


#추후에left조인(by: user_id)을 이용하고 fillna(0)을 이용

(8268, 31) (8268, 31)
